## Funciones comunes

In [0]:
from pyspark.sql.functions import current_timestamp

def add_ingestion_date(input_df):
    output_df=input_df.withColumn("ingestion_time",current_timestamp())
    return output_df

In [0]:
def overwrite_partition(input_df,db_name,table_name,partition_column):
    for item_list in input_df.select(f"{partition_column}").distinct().collect():
        if(spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
            spark.sql(f"ALTER TABLE {db_name}.{table_name} DROP IF EXISTS PARTITION({partition_column}='{item_list[partition_column]}')")


In [0]:
from delta.tables import *

def merge_delta_lake(input_df,db_name,table_name,folder_path,merge_condition,partition_column):
    if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
        deltaTable = DeltaTable.forPath(spark, f"{folder_path}/{table_name}")
        deltaTable.alias("target").merge(
            input_df.alias("source"),
            f"{merge_condition}",
        ).whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll().execute()

    else:
        input_df.write.mode("overwrite").partitionBy(f"{partition_column}").format("delta").saveAsTable(f"{db_name}.{table_name}")